![Example grocery store](grocery_store.jpg)


In the bustling world of retail, especially in grocery stores, understanding consumer preferences and shopping habits is pivotal for business optimization. Imagine a local grocery chain seeking to enhance customer satisfaction and drive sales. By analyzing transactional data from two of its 24-hour stores, each with unique data storage practices due to the store owners' diverse backgrounds, the chain aims to uncover hidden patterns in consumer behavior. The goal here is multifaceted: optimizing store layouts, tailoring marketing strategies, and efficiently managing inventory to align with customer preferences. For instance, if analysis reveals that customers frequently purchase certain products together or show a preference for shopping at specific hours, the store can adjust its marketing efforts and stock levels accordingly. Additionally, understanding seasonal trends and weekly fluctuations in sales helps in planning promotions and staffing. Acting as the retail data scientist for this chain, you'll leverage detailed retail data to drive strategic decisions, ultimately enhancing customer experience and boosting the store's performance. 

`grocery_data1.csv` and `grocery_data2.csv` contain detailed records of grocery transactions from these two stores with the same column names:

| Variable         | Description                                           |
|------------------|-------------------------------------------------------|
| `CustomerID`     | Unique identifier for each customer                   |
| `DateRaw`        | Raw date of the transaction                           |
| `Time`           | Time of the transaction                               |
| `TransactionID`  | Unique identifier for each transaction                |
| `ProductName`    | Name of the product purchased                         |
| `PriceUSD`       | Price of the product in US dollars                    |
| `Quantity`       | Number of the product purchased                     |
| `PaymentMethod`  | Payment type used for the transaction               |
| `Category`       | Category of the product                               |


### Import the required Libraries

In [17]:
# Import packages
library(dplyr)
library(lubridate)
library(readr)

In [20]:
# Load and parse the time of the grocery datasets
grocery_data1 <- read_csv('grocery_data1.csv') %>% 
  mutate(Date = mdy(DateRaw))
grocery_data2 <- read_csv('grocery_data2.csv') %>% 
  mutate(Date = dmy(DateRaw))
grocery_data <- grocery_data1 %>% 
  bind_rows(grocery_data2)

# Calculate TotalSaleUSD = PriceUSD * Quantity
grocery_data <- grocery_data %>%
  mutate(TotalSaleUSD = PriceUSD * Quantity)

head(grocery_data,3)

Rows: 2619 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (4): DateRaw, ProductName, PaymentMethod, Category
dbl  (4): CustomerID, TransactionID, PriceUSD, Quantity
time (1): Time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 2581 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (4): DateRaw, ProductName, PaymentMethod, Category
dbl  (4): CustomerID, TransactionID, PriceUSD, Quantity
time (1): Time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


CustomerID,DateRaw,Time,TransactionID,ProductName,PriceUSD,Quantity,PaymentMethod,Category,Date,TotalSaleUSD
<dbl>,<chr>,<time>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<date>,<dbl>
41,"June 28, 2023",20:00:00,2,Apples,5.64,5,Cash,Produce,2023-06-28,28.20
170,"August 18, 2023",06:00:00,3,Apples,17.92,1,Mobile Payment,Produce,2023-08-18,17.92
86,"August 18, 2023",09:00:00,4,Pasta,19.14,2,Mobile Payment,Grains,2023-08-18,38.28


In [21]:
# Calculate the time since each customer purchased each ProductName
grocery_data_grouped <- grocery_data %>%
  arrange(CustomerID, Date) %>%
  group_by(CustomerID, ProductName) %>%
  mutate(
    DaysSinceLastPurchase = as.numeric(c(0, diff(Date))),
    Week = week(Date),
    Year = year(Date),
    Hour = hour(Time)
  ) %>%
  ungroup()

tail(grocery_data_grouped, 2)

CustomerID,DateRaw,Time,TransactionID,ProductName,PriceUSD,Quantity,PaymentMethod,Category,Date,TotalSaleUSD,DaysSinceLastPurchase,Week,Year,Hour
<dbl>,<chr>,<time>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
179,17 August 2023,20:00:00,2476,Milk,17.87,4,Mobile Payment,Dairy,2023-08-17,71.48,60,33,2023,20
179,31 August 2023,18:00:00,344,Rice,5.57,4,Credit Card,Grains,2023-08-31,22.28,27,35,2023,18


In [22]:
# Calculate weekly figures for TotalSaleUSD
weekly_sales <- grocery_data_grouped %>%
  group_by(Week, Year) %>%
  summarize(WeeklyTotalSaleUSD = sum(TotalSaleUSD)) %>% 
  arrange(Week, Year) %>% 
  ungroup()

weekly_sales

`summarise()` has grouped output by 'Week'. You can override using the
`.groups` argument.


Week,Year,WeeklyTotalSaleUSD
<dbl>,<dbl>,<dbl>
22,2023,3907.00
23,2023,12189.07
24,2023,11111.63
25,2023,11005.97
26,2023,13733.57
27,2023,11954.92
28,2023,12286.75
29,2023,12652.53
30,2023,12617.20


In [23]:
# What week of the year during the time period had the smallest absolute deviation in sales value compared to the mean WeeklyTotalSaleUSD?
mean(weekly_sales$WeeklyTotalSaleUSD)
weekly_sales %>% 
  mutate(Diff = abs(WeeklyTotalSaleUSD - mean(WeeklyTotalSaleUSD))) %>% 
  arrange(Diff)
smallest_sales_deviation <- 24


[1] 11442.88

Week,Year,WeeklyTotalSaleUSD,Diff
<dbl>,<dbl>,<dbl>,<dbl>
24,2023,11111.63,331.2514
25,2023,11005.97,436.9114
27,2023,11954.92,512.0386
34,2023,12144.57,701.6886
31,2023,12154.19,711.3086
23,2023,12189.07,746.1886
28,2023,12286.75,843.8686
30,2023,12617.20,1174.3186
29,2023,12652.53,1209.6486


In [24]:
# Calculate hourly figures for TotalSaleUSD
hourly_sales <- grocery_data_grouped %>%
  group_by(Hour) %>%
  summarize(HourlyTotalSaleUSD = sum(TotalSaleUSD)) %>% 
  ungroup() %>%
  arrange(desc(HourlyTotalSaleUSD))

In [25]:
# What hour of the day (as a number on the 24 hour scale) had the most HourlyTotalSaleUSD?
most_hourly_sales <- 22

In [26]:
# Examine Cornflakes purchases for CustomerID 107
grocery_data_grouped %>% 
  filter(CustomerID == 107) %>% 
  filter(ProductName == "Cornflakes") %>%
  arrange(desc(DaysSinceLastPurchase))

CustomerID,DateRaw,Time,TransactionID,ProductName,PriceUSD,Quantity,PaymentMethod,Category,Date,TotalSaleUSD,DaysSinceLastPurchase,Week,Year,Hour
<dbl>,<chr>,<time>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
107,31 August 2023,05:00:00,1056,Cornflakes,12.60,1,Credit Card,Cereal,2023-08-31,12.60,40,35,2023,5
107,22 July 2023,05:00:00,3188,Cornflakes,18.24,1,Debit Card,Cereal,2023-07-22,18.24,6,29,2023,5
107,16 July 2023,17:00:00,5015,Cornflakes,4.30,3,Debit Card,Cereal,2023-07-16,12.90,0,29,2023,17


In [27]:
# How many days went by between the first and second and the second and third transactions of the three purchases of Corn Flakes by CustomerID 107?
cornflakes_days <- c(6, 40)